Hello! Here is some documented sample code that is executable (simply hit shift enter in each cell to run its contents).

I've included a piece of code that is tested and some visualisations...


# Stochastic Charging Algorithm

## Brief Rationale and background for the charging method
A nanograin’s charge varies stochastically with time and therefore in each timestep the number of electrons/ions collected by the grain must be estimated. 

<img src=/summary_currents_schematic.png width="250">

Instead of modelling the inter-arrival times of plasma particles using an exponential distribution, the Poisson distribution can be used to model the number of charged particles being collected by the grain in a given timestep. The type of particle is handled by the algorithm as described below, involving Poisson and binomial probability distributions and weighting the various currents appropriately.

## Theory
[Hsu et al. (2011)](https://doi.org/10.1029/2011JA016488) introduce a Poisson process approach, which is further developed in
the work of [Hsu et al. (2018)](https://doi.org/10.1126/science.aat3185). They developed their procedure independently of [Cui & Goree
(1994)](https://doi.org/10.1109/27.279018) but reach a similar approach. Their method can be summarised as 
    - (i) Generate a random number to determine how many events occur during the timestep using a Poisson probability distribution
    - (ii) Decide the type of charging events that occurs according to the relative strengths of the various currents.


## Simple example

## Unit test

## Batch test



In [1]:
#Stand-alone tree walking program (to traverse the binary tree)
# Multiple current with R set at each level for testing purposes (rather than allowing randomness) 

import numpy as np
from scipy.stats import binom
from binarytree import Node # see https://pypi.org/project/binarytree/ and https://github.com/joowani/binarytree/blob/master/binarytree/__init__.py
# Note that I've saved a backup of the current binarytree package source code in /Users⁩/mm9684⁩/Documents⁩/PhD_related⁩/binarytree-master_backup_from_github_joowani


def nodify(k):
    # Use a function to sort out the string concatenating and then transforming into a float
    # Node method cannot handle non-numbers so I need to hack a way to display the plasma electron, plasma ion, and photoelectron event numbers at each level
    # k = [k_e, k_i, k_nu]
    return float(str(k[0]) + "." + str(k[1]) + str(k[2]) ) # note that if k_e=1, k_i=2, k_nu = 0 then it will read as "1.2" rather than "1.20"


def binomial_prob_multiI(k_arr, R):
    # Multiple currents

    # Use the drawn uniform number to see how the charging events are split between the parent's 2 child sub-steps
    # I need to unpack the total number of charging events from the list passed into this function:
    N_e = k_arr[0]
    N_i = k_arr[1]
    N_nu = k_arr[2]

    #print(f"N_e = {N_e}, N_i = {N_i}, N_nu = {N_nu}")
    if N_e == 0 and N_i == 0 and N_nu == 0:
        print("WARNING: all N_e and N_i and N_nu = 0 !!")
    elif N_e == 1 and N_i == 1 and N_nu == 1: # if there happens to be one electron and one ion charging event and one photoemission event, it should be equally probable to pick either of the 3
        #print("In special case [1,1,1]")
        k_ctr = [99999999999999, 99999999999999, 99999999999999] # Initialize variable: use redonkulus value rather than NaN, otherwise Node() complains!
        if R <= (1.0/3.0):
            k_ctr[0] = 1; k_ctr[1] = 0; k_ctr[2] = 0
        elif (1.0/3.0) < R <= (2.0/3.0):
            k_ctr[0] = 0; k_ctr[1] = 1; k_ctr[2] = 0
        else: # if R lies between 2/3 and 1
            k_ctr[0] = 0; k_ctr[1] = 0; k_ctr[2] = 1
    else:
        #print(f"In else clause, R = {R}")
        #R = np.random.uniform(low=0.0, high=1.0) # draw a random number from Uniform probability distribution
        #print("binomial_prob R = ", R, "N = ", N)
        # Initialise 2-element array to store result for [I_e, I_i] number of charging events
        #k_ctr = [0, 0] # use zero rather than NaN, otherwise Node() complains!
        k_ctr=[99999999999999, 99999999999999, 99999999999999] # use an integer value that would get clearly flagged up if there's a bug in my code
        
        # Check where R falls in the weighted nested Binomial probability distribution
        # Note that because there is more than one type of current, I need to concurrently work out the respective probabilities for each type
        # Therefore I need to define separate counters k_e_ctr, k_i_ctr, k_nu_ctr
        
        ##First define plasma electron current probability variables
        start_pt_e = 0.0 # define a variable to store the start point of probability distribution range for I_e
        k_e_ctr = 0
        # Define the end point for the first partition (for X_e = 0) using prob mass function to calculate binomial probability for k_e=0, making sure to weight it appropriately
        end_pt_e = (N_e/(N_e+N_i+N_nu))*binom.pmf(k=k_e_ctr, n=N_e, p=0.5)
        
        #Rather than generating the full Binomial probability ranges, instead perform "Just in time" calculations, so that unnecessary calculations are avoided
        # First check whether it's an I_e type event:
        
        # Bug fix: if there happens to be one electron charging event and R falls inside of the k_e events bin then R must fall inside the k_e=1 event prob range
        if N_e == 1 and R < end_pt_e: 
            #print("In if N_e == 1")
            k_ctr[0]=1; k_ctr[1]=0; k_ctr[2]=0
            return(k_ctr)
            
        while ((R >= end_pt_e) and (k_e_ctr < N_e)):
            #print("In while ((R >= end_pt_e) and (k_e_ctr < N_e)) ")
            k_e_ctr+=1 #increment to the next prob range
            start_pt_e = end_pt_e # move start_pt to the next prob range bin's start (which is the end of the previous prob range's bin)
            end_pt_e += (N_e/(N_e+N_i+N_nu))*binom.pmf(k=k_e_ctr, n=N_e , p=0.5) # add on the appropriately weighted prob to update the end point of the probability bin
            #print("R=", R, "Prob range for k_e = ", k_e_ctr, " is ", start_pt_e , "to ", end_pt_e )
        # Store the number of I_e charging events as the 0th element in the list of k events:
        k_ctr[0] = k_e_ctr # note that if R doesn't lie within the I_e weighted range, then k_e_ctr is erroneously set to be N_e but it gets corrected below
        k_ctr[1] = 0 # If an electron event has been chosen then the ion event must be zero
        k_ctr[2] = 0 # If an electron event has been chosen then the photoelectron event must be zero
        #print(f"k_ctr = {k_ctr}")
        
        # Then check whether it's an I_i type event (only do this if R has not been found to lie inside I_e probability ranges)
        if R > end_pt_e: # only need to check the I_i prob ranges if R has proven to lie outside the I_e prob ranges
            #print("In if R > end_pt_e ")
            k_ctr[0] = 0 #if the code enters this if block then know that the plasma electron current has not been picked and so I need to overwrite with zero
            k_ctr[2] = 0 # also ensure that the photoelectron event is zero in the case of an ion event
                        
            # To save memory, only initialise variables if they are actually going to be used!
            # Define a variable to store the start point of probability distribution range for I_i current
            start_pt_i = N_e/(N_e+N_i+N_nu) # this coincides with the end point of the I_e current
            k_i_ctr = 0
            # Define the end point for the first partition (for X_i = 0) using prob mass function to calculate binomial probability for k_i=0
            end_pt_i = start_pt_i + (N_i/(N_e+N_i+N_nu))*binom.pmf(k=k_i_ctr, n=N_i, p=0.5)
            
            # Bug fix: if there happens to be one ion charging event and R does not fall inside any of the k_e events then R must fall inside the k_i event prob range
            if N_i == 1 and R < end_pt_i: 
                #print("In if N_i == 1")
                k_ctr[0]=0; k_ctr[1]=1; k_ctr[2]=0
                return(k_ctr)
            
            #print(f"k_ctr = {k_ctr}")
            #print(f"start_pt_i = {start_pt_i}, end_pt_i = {end_pt_i}")
            while ((R >= end_pt_i) and (k_i_ctr < N_i)):
                #print(f"In while ((R >= end_pt_i) and (k_i_ctr < N_i)) ")
                k_i_ctr+=1 #increment to the next prob range
                start_pt_i = end_pt_i # move start_pt to the next prob range bin's start (which is the end of the previous prob range's bin)
                end_pt_i += (N_i/(N_e+N_i+N_nu))*binom.pmf(k=k_i_ctr, n=N_i , p=0.5) # add on the appropriately weighted prob to update the end point of the probability bin
                #print("R=", R, "Prob range for k_i = ", k_i_ctr, " is ", start_pt_i , "to ", end_pt_i )
            # Store the number of I_i charging events as the 1st element in the list of k events:
            k_ctr[1] = k_i_ctr
            #print(f"k_ctr = {k_ctr}")
       ##print("and while_ctr = ", while_ctr)
       #print("R lies in bin where k = ", k_ctr)
            # Then check whether it's an I_nu type event (only do this if R has not been found to lie inside I_e or I_i probability ranges)
            if R > end_pt_i: # only need to check the I_i prob ranges if R has proven to lie outside the I_e prob ranges (note the indentation)
                #print("In if R > end_pt_e ")
                k_ctr[0] = 0 #if the code enters this if block then know that the plasma electron current has not been picked and so I need to overwrite with zero
                k_ctr[1] = 0 # also ensure that the plasma ion current event is zero in the case of a photoelectron event
               
                # To save memory, only initialise variables if they are actually going to be used!
                # Define a variable to store the start point of probability distribution range for I_nu current
                start_pt_nu = (N_e+N_i)/(N_e+N_i+N_nu) # this coincides with the end point of the (I_e + I_i) currents
                k_nu_ctr = 0
                # Define the end point for the first partition (for X_i = 0) using prob mass function to calculate binomial probability for k_i=0
                end_pt_nu = start_pt_nu + (N_nu/(N_e+N_i+N_nu))*binom.pmf(k=k_nu_ctr, n=N_nu, p=0.5)
                 # Bug fix: if there happens to be one ion charging event and R does not fall inside any of the k_e events then R must fall inside the k_i event prob range
                if N_nu == 1 and R < end_pt_nu: 
                    #print("In if N_nu == 1")
                    k_ctr[0]=0; k_ctr[1]=0; k_ctr[2]=1
                    return(k_ctr)
                
                #print(f"k_ctr = {k_ctr}")
                #print(f"start_pt_nu = {start_pt_nu}, end_pt_nu = {end_pt_nu}")
                while ((R >= end_pt_nu) and (k_nu_ctr < N_nu)):
                    #print(f"In while ((R >= end_pt_nu) and (k_nu_ctr < N_nu)) ")
                    k_nu_ctr+=1 #increment to the next prob range
                    start_pt_nu = end_pt_nu # move start_pt to the next prob range bin's start (which is the end of the previous prob range's bin)
                    end_pt_nu += (N_nu/(N_e+N_i+N_nu))*binom.pmf(k=k_nu_ctr, n=N_nu , p=0.5) # add on the appropriately weighted prob to update the end point of the probability bin
                    #print("R=", R, "Prob range for k_i = ", k_i_ctr, " is ", start_pt_i , "to ", end_pt_i )
                # Store the number of I_i charging events as the 1st element in the list of k events:
                k_ctr[2] = k_nu_ctr
                #print(f"k_ctr = {k_ctr}")
        
    return k_ctr

def walker(value_arr, level=0, tree=Node(0), level_1_switch=1): 
    print("Inside walker()")
    # It was necessary to pass the level_1_switch to the walker() function explicitly for my unit test toy example
    
    if ( (value_arr[0]==1 and value_arr[1]==0 and value_arr[2]==0) or (value_arr[0] == 0 and value_arr[1] == 1 and value_arr[2]==0) or (value_arr[0] == 0 and value_arr[1] == 0 and value_arr[2]==1) or
        (value_arr[0]==0 and value_arr[1]==0 and value_arr[2]==0)): # base case (leaf nodes have no children)
        print("Base case")
        tree.left = None 
        tree.right = None
        return [(value_arr, level)], tree, level_1_switch # store the leaf node value (ie k=0 or k=1) and its corresponding level as tuple-element of list
    
    print(f"level_1_switch = {level_1_switch}")
    #    # Unit test (rather than allowing R to be random, choose R so that can check get the expected output)
    if level == 0:
        #print("here 0 ")
        R = 0.47
    elif level == 1 and level_1_switch ==0 :
        R = 0.3
        level_1_switch = 1
    elif level_1_switch == 1 and level_1_switch ==1:
        R = 0.45
        level_1_switch = 0
    elif level == 2 and level_1_switch==0:
        R = 0.92
    elif level == 3:
        R = 0.1
    elif level == 4:
        R = 0.67
    elif level_1_switch == 1: # in order to make the code enter the level == 1 subblock twice, flip the switch on
        R = 0.45    #  there needs to be a double split happening at level 1, i.e. k_e = 2 needs to split as does k_e=1, k_i=2
    else:
        R = 1000 # I need to assign a UNUSED value to R, otherwise Python complains about R being referenced before assignment
   
    # Use binomial prob to determine how the number of charging events are split across 1st and 2nd half of step
    substep1_arr = binomial_prob_multiI(value_arr, R)  # value holds the list of [k_e, k_e] number of charging events
    #substep2_arr = np.array(value_arr) - np.array(substep1_arr) # Note that because value_arr is a numpy array as is substep1, I can do elementwise subtraction (ie treat the electron and ion counters separately)
    substep2_arr = [99999999999999,99999999999999,99999999999999] # intialise list (rather than mixing np array type)
    substep2_arr[0] = value_arr[0] - substep1_arr[0]
    substep2_arr[1] = value_arr[1] - substep1_arr[1]
    substep2_arr[2] = value_arr[2] - substep1_arr[2]

    print(f"LEVEL = {level} R={R},  substep1 = {substep1_arr}")
    #print(f"substep2 = {substep2_arr}")
    # Define the left and child nodes:
    tree.left = Node(nodify(substep1_arr)) # use my hacky nodify function as Node cannot handle non-numbers (e.g. arrays)
    tree.right = Node(nodify(substep2_arr))
    
    a_arr, tree_l, level_1_switch = walker(substep1_arr, level+1, tree.left, level_1_switch) # recursive call to left child
    b_arr, tree_r, level_1_switch = walker(substep2_arr, level+1, tree.right, level_1_switch) # recursive call to right child
    
    
    #print(f"walker level{level} called with value {value_arr} splits into {substep1_arr} and {substep2_arr} as R = {R}")
    #print(f" left subtree a_arr = {a_arr}, right subtree b_arr = {b_arr}")
    # Reconfigure the binary tree to update:
    tree.left = tree_l
    tree.right = tree_r
    return a_arr + b_arr, tree, level_1_switch #concatenate list of lists (a_arr+b_arr) and also return tree


#N_init = [2,1] 
N_init = [3,2,0] # initialise k_e = 3, k_i = 2, k_nu=0
lev = 0
lev_1_switch = 0
leaf_nodes, tree, lev_1_switch = walker(N_init, level=lev, tree=Node(nodify(N_init)), level_1_switch=0)
print("\nFinally :")
print("Result: ", leaf_nodes) # leaf_nodes lists tuples of (value, level) ie it lists tuples of (num_of_charging_event, l-index of 1/2^l substep)
print(tree)

# Use the results from the tree_walker to traverse a (toy dummy) step:
print("Traversing step")

delta_t = 1.0 # some arbitrary initial timestep size
t = 0.0 # dummy time variable 
Q = 0.0 # dummy charge variable
t_arr = [t]
Q_arr = [Q]
    
for (Q_sub, level) in leaf_nodes:
    print ("leaf_nodes: ", leaf_nodes)
    print(f"Charging event of {Q_sub} at level {level}")
    t = t + delta_t/(2**level)
    Q = Q - Q_sub[0] # "plasma electron current"
    Q = Q + Q_sub[1] # "plasma ion current"
    Q = Q + Q_sub[2] # "photoelectron current" (think it's in plus sense like plasma ion??)

    t_arr.append(t)
    Q_arr.append(Q)

print(f"Step results, t = {t_arr }; Q = {Q_arr}")

Inside walker()
level_1_switch = 0
LEVEL = 0 R=0.47,  substep1 = [2, 0, 0]
Inside walker()
level_1_switch = 0
LEVEL = 1 R=0.3,  substep1 = [1, 0, 0]
Inside walker()
Base case
Inside walker()
Base case
Inside walker()
level_1_switch = 1
LEVEL = 1 R=0.45,  substep1 = [0, 0, 0]
Inside walker()
Base case
Inside walker()
level_1_switch = 0
LEVEL = 2 R=0.92,  substep1 = [0, 2, 0]
Inside walker()
level_1_switch = 0
LEVEL = 3 R=0.1,  substep1 = [0, 0, 0]
Inside walker()
Base case
Inside walker()
level_1_switch = 0
LEVEL = 4 R=0.67,  substep1 = [0, 1, 0]
Inside walker()
Base case
Inside walker()
Base case
Inside walker()
Base case

Finally :
Result:  [([1, 0, 0], 2), ([1, 0, 0], 2), ([0, 0, 0], 2), ([0, 0, 0], 4), ([0, 1, 0], 5), ([0, 1, 0], 5), ([1, 0, 0], 3)]

       _____3.2_____
      /             \
   _2.0_           _1.2_____________________
  /     \         /                         \
1.0     1.0     0.0            _____________1.2_
                              /                 \
   